In [1]:
# Import basic libraries to handle data and build the model
import zipfile
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets, transforms
from torch.utils.data import random_split

In [2]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 20.07 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.is_available()

True

In [5]:
os.chdir("/content/Qualcomm-DL-Hackathon/train")
extract_dir = "/content/Qualcomm-DL-Hackathon/train/"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("images part-1.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
with zipfile.ZipFile("images part-2.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/Qualcomm-DL-Hackathon/train/


In [6]:
os.rename('/content/Qualcomm-DL-Hackathon/train/images part-1', '/content/Qualcomm-DL-Hackathon/train/images')

In [7]:
import shutil
for x in os.listdir('/content/Qualcomm-DL-Hackathon/train/images part-2'):
  source_path = os.path.join('/content/Qualcomm-DL-Hackathon/train/images part-2', x)
  shutil.move(source_path, '/content/Qualcomm-DL-Hackathon/train/images/')

In [8]:
#shutil.rmtree('/content/Qualcomm-DL-Hackathon/train/images')
shutil.rmtree('/content/Qualcomm-DL-Hackathon/train/images part-2')

In [9]:
from PIL import Image
os.getcwd()

'/content/Qualcomm-DL-Hackathon/train'

In [10]:
#reading images to check size
count = 0
for x in os.listdir('/content/Qualcomm-DL-Hackathon/train/images/'):
  img = Image.open('/content/Qualcomm-DL-Hackathon/train/images/'+x)
  count+=1
count

2352

In [11]:
img.size

(224, 224)

In [12]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [13]:
train_data.shape

(1646, 2)

In [14]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.shape

(706, 1)

In [15]:
data=pd.DataFrame(train_data)
#data=data.sample(frac=1)
data

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0
...,...,...
1641,1638.jpg,0
1642,1095.jpg,0
1643,1130.jpg,0
1644,1294.jpg,0


In [16]:
def split_data(data,ratio):
    last=int(len(data)*ratio)
    return data[:last], data[last:]
test,train=split_data(data,.25)
train.shape, test.shape

((1235, 2), (411, 2))

In [17]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
from PIL import Image
size=224
batch_size = 32
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, -1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [18]:
batch_size=32
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.Resize((size, size)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ,
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((size, size)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ,
])

custom_dataset_train = CustomDataset(train, transform=transform_train)
data_loader_train = DataLoader(custom_dataset_train, batch_size=batch_size, shuffle=True)

custom_dataset_test = CustomDataset(test, transform=transform)
data_loader_test = DataLoader(custom_dataset_test, batch_size=1, shuffle=False)

In [45]:
class FineTunedVGG(nn.Module):
    def __init__(self, num_classes, input_size=224):
        super(FineTunedVGG, self).__init__()
        self.vgg = models.vgg16(pretrained=True)

        self.st = 8
        self.blocks = []
        for param in self.vgg.parameters():
            param.requires_grad = False

        x = torch.randn(1, 3, input_size, input_size)
        for idx, layer in enumerate(self.vgg.features):
            if isinstance(layer, nn.Conv2d):
                x = layer(x)
                if idx in [12, 22, 32]:
                    self.blocks.append(x)


        for idx, block in enumerate(self.blocks):
            filters = block.size(1)
            depthwise_conv = nn.Conv2d(filters, filters, kernel_size=3, padding=1, groups=filters)
            depthwise_sep_conv = nn.Conv2d(filters, 128, kernel_size=1, padding=0)
            bn = nn.BatchNorm2d(128)
            pooled_block = nn.MaxPool2d(kernel_size=self.st, stride=self.st)
            self.st = self.st // 2
            self.blocks[idx] = nn.Sequential(depthwise_conv, depthwise_sep_conv, bn, pooled_block)

        self.vgg.add_module('ConcatenatedBlocks', nn.Sequential(*self.blocks))

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.vgg(x)
#         x=self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = FineTunedVGG(num_classes=2, input_size=224)

In [46]:
os.chdir("/content/Qualcomm-DL-Hackathon/train/images")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [47]:
num_epochs = 10
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
running_loss = 0.0
total = 0
correct = 0
i=0

for epoch in range(num_epochs):
    for images, labels in data_loader_train:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #print(i,end=' ')
        i+=1
        # Track the loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader_train)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print("Training finished.")

Epoch [1/10], Loss: 0.8187, Accuracy: 65.91%
Epoch [2/10], Loss: 1.3037, Accuracy: 71.82%
Epoch [3/10], Loss: 1.7887, Accuracy: 73.77%
Epoch [4/10], Loss: 2.2485, Accuracy: 75.04%
Epoch [5/10], Loss: 2.6973, Accuracy: 76.13%
Epoch [6/10], Loss: 3.1289, Accuracy: 76.91%
Epoch [7/10], Loss: 3.6037, Accuracy: 77.20%
Epoch [8/10], Loss: 4.0395, Accuracy: 77.68%
Epoch [9/10], Loss: 4.5011, Accuracy: 77.95%
Epoch [10/10], Loss: 4.9649, Accuracy: 78.08%
Training finished.


In [48]:
# Validation phase
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
     for inputs, labels in data_loader_test:
         inputs=inputs.to(device)
         labels=labels.to(device)
         outputs = model(inputs)
         loss = criterion(outputs, labels)
         val_loss += loss.item()
         _, predicted = torch.max(outputs, 1)
         total += labels.size(0)
         correct += (predicted == labels).sum().item()

print(f"Validation Loss: {val_loss/len(data_loader_test)}, Accuracy: {100 * correct / total}%")

Validation Loss: 0.25559798725371674, Accuracy: 89.29440389294403%


In [ ]:
# Generate predictions for test images
model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Print or save the predictions
print(predictions)